## Basic Pipeline

In [ ]:
# Retreiving
# Indexing - Elasticsearch, TF-IDF 
# Querying

In [ ]:
import minisearch


ms = minisearch.Index()

minsearch

### Prpmpt

In [ ]:
from llama_index.core import PromptTemplate

template = (
    "We have provided context information below. \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "Given this information, please answer the question: {query_str}\n"
)
qa_template = PromptTemplate(template)

# you can create text prompt (for completion API)
prompt = qa_template.format(context_str=..., query_str=...)

## Dataset

Amod/mental_health_counseling_conversations

In [1]:
import datasets

dataset = datasets.load_dataset("Amod/mental_health_counseling_conversations")

print(dataset)

c:\Users\h4has\anaconda3\envs\rag\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['Context', 'Response'],
        num_rows: 3512
    })
})


In [9]:
print(dataset['train'])
print(type(dataset['train']))

Dataset({
    features: ['Context', 'Response'],
    num_rows: 3512
})
<class 'datasets.arrow_dataset.Dataset'>


In [2]:
for data in dataset['train']:
    print(data)
    break

{'Context': "I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.\n   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.\n   How can I change my feeling of being worthless to everyone?", 'Response': "If everyone thinks you're worthless, then maybe you need to find new people to hang out with.Seriously, the social context in which a person lives is a big influence in self-esteem.Otherwise, you can go round and round trying to understand why you're not worthless, then go back to the same crowd and be knocked down again.There are many inspirational messages you can find in social media. \xa0Maybe read some of the ones which state that no person is worthless, and that everyone has a good purpose to their life.Also, since our culture is so saturated with the belief that if someone doesn't feel good about themselves that this is someho

## Model

In [7]:
model = 'meta-llama/Meta-Llama-3.1-8B-Instruct'
# model = 't5-small'

In [ ]:
from transformers import LlamaTokenizer, Llama

tokenizer = LlamaTokenizer.from_pretrained(model)
model = LlamaForCausalLM.from_pretrained(model)



In [3]:
# from transformers import T5Tokenizer, T5ForConditionalGeneration

# tokenizer = T5Tokenizer.from_pretrained("t5-small")
# model = T5ForConditionalGeneration.from_pretrained("t5-small")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
c:\Users\h4has\anaconda3\envs\rag\lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\h4has\.cache\huggingface\hub\models--t5-small. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.c

In [5]:
query = "What is mental health?"
inputs = tokenizer(query, return_tensors="pt")
print(inputs)

{'input_ids': tensor([[ 363,   19, 2550,  533,   58,    1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}


In [6]:

outputs = model.generate(**inputs)
print(outputs)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

tensor([[   0, 2751,  229, 2550,  533,   58,    1]])
Was ist mental health?


c:\Users\h4has\anaconda3\envs\rag\lib\site-packages\transformers\generation\utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
